In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import igraph as ig
import leidenalg as la
import seaborn as sns
import time
import re
import networkx as nx
from operator import itemgetter
import warnings
import matplotlib.cm as mcm
import community
from tqdm.notebook import tqdm
from community import community_louvain as cm
# warnings.filterwarnings('ignore')

In [8]:
path_base = r"C:\Users\gianluca.nogara\Desktop\Repo\Vaccines_Discussion_Italy\Italian"

## Retweets

In [13]:
tweets = pd.read_parquet(path_base +r'\files\tweets\tweets.parquet', engine="pyarrow")
retweets = tweets[tweets["rt_created_at"].notna()]

# NETWORK

In [14]:
retweets_graph = nx.from_pandas_edgelist(retweets,'user_screen_name','rt_user_screen_name', create_using=nx.DiGraph())
print(nx.info(retweets_graph))

DiGraph with 216216 nodes and 1409724 edges


C:\Users\gnogara.inst\AppData\Local\Temp\ipykernel_14868\3591911845.py:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(retweets_graph))


In [15]:
degree_dict = dict(retweets_graph.degree(retweets_graph.nodes()))
sorted_degree = sorted(degree_dict.items(), key=itemgetter(1), reverse=True)

In [16]:
in_degree_dict = dict(retweets_graph.in_degree(retweets_graph.nodes()))
sorted_in_degree = sorted(in_degree_dict.items(), key=itemgetter(1), reverse=True)

In [17]:
out_degree_dict = dict(retweets_graph.out_degree(retweets_graph.nodes()))
sorted_out_degree = sorted(out_degree_dict.items(), key=itemgetter(1), reverse=True)

## Communities with Louvain modularity

In [18]:
retweets_graph_und = nx.from_pandas_edgelist(retweets,'user_screen_name','rt_user_screen_name')
print(nx.info(retweets_graph_und))

Graph with 216216 nodes and 1394787 edges


C:\Users\gnogara.inst\AppData\Local\Temp\ipykernel_14868\896874219.py:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(retweets_graph_und))


In [19]:
best_part = cm.best_partition(retweets_graph_und, random_state=11)

In [16]:
lst_name = []
lst_index = []
for i in tqdm(best_part):
    lst_name.append(i)
    lst_index.append(best_part.get(i))
pd.DataFrame(list(zip(lst_name, lst_index)), columns=["name", "community"]).to_csv("C:/Users/gianl/Desktop/Gi/Supsi/MartaLauraGiada/Laura/Files/louvain.csv", encoding="utf-8", line_terminator="\n", index=False)

In [20]:
import networkx.algorithms.community as nx_comm
nx_comm.modularity(retweets_graph_und, nx_comm.label_propagation_communities(retweets_graph_und))

0.46293066238257996

# Eigenvector Centrality

In [21]:
ev_centrality = nx.eigenvector_centrality(retweets_graph_und)

In [22]:
lst_ev = [ev_centrality[node] for node in ev_centrality]
lst_name = [node for node in ev_centrality]
df_ev_centrality = pd.DataFrame(list(zip(lst_name,lst_ev)), columns=["name","ev"])
df_ev_centrality

,name,ev
0,planetpaul65,7.765983e-04
1,isentinelli,1.290955e-04
2,Grace_1207,3.794276e-04
3,LidaSezOlbia,5.918241e-03
4,MarisaMinervini,1.290487e-07
...,...,...
216211,legatumori_to,8.288815e-10
216212,ExmoorOn,2.803193e-06
216213,Dado46544020,1.207017e-05
216214,BernardMalcom2,7.213236e-05


In [28]:
path = r"C:\Users\gianl\Desktop\Gi\Supsi\Vaccines_Discussion_Italy\Laura\Files"
df_ev_centrality.sort_values(by='ev', ascending=False, inplace=True)
df_ev_centrality.to_csv(path + r"\user_ev_centrality.csv",
                        line_terminator="\n",
                        index=False,
                        encoding="utf-8")

# Degree centrality

In [23]:
degree_centrality = nx.degree_centrality(retweets_graph_und)

In [24]:
lst_deg = [degree_centrality[node] for node in degree_centrality]
lst_name = [node for node in degree_centrality]
df_dg_centrality = pd.DataFrame(list(zip(lst_name,lst_deg)), columns=["name","deg_cen"])
df_dg_centrality

,name,deg_cen
0,planetpaul65,0.000532
1,isentinelli,0.000726
2,Grace_1207,0.000194
3,LidaSezOlbia,0.004764
4,MarisaMinervini,0.000005
...,...,...
216211,legatumori_to,0.000005
216212,ExmoorOn,0.000005
216213,Dado46544020,0.000005
216214,BernardMalcom2,0.000005


In [32]:
df_dg_centrality.sort_values(by='deg_cen', ascending=False, inplace=True)
df_dg_centrality.to_csv(path + r"\user_deg_centrality.csv",
                        line_terminator="\n",
                        index=False,
                        encoding="utf-8")

# Betwenness centrality

In [28]:
betweenness_centrality = nx.betweenness_centrality(retweets_graph_und, k=1000)
print("Checkpoint")
lst_bet = [betweenness_centrality[node] for node in betweenness_centrality]
lst_name = [node for node in betweenness_centrality]
df_bet_centrality = pd.DataFrame(list(zip(lst_name,lst_bet)), columns=["name","bet"])
df_bet_centrality

KeyboardInterrupt: 

In [ ]:
df_bet_centrality.head()

In [ ]:
df_bet_centrality.sort_values(by='bet', ascending=False, inplace=True)
df_bet_centrality.to_csv(r"C:\Users\gianl\Desktop\Gi\Supsi\Vaccines_Discussion_Italy\Italian\Files\user_bet_centrality.csv",
                        line_terminator="\n",
                        index=False,
                        encoding="utf-8")